In [ ]:
#%pip install "autogen-ext[magentic-one,openai]" 
#%pip install gradio
# If using the MultimodalWebSurfer, you also need to install playwright dependencies:


## Magentic One Overview

Magentic-One is a generalist multi-agent system for solving open-ended web and file-based tasks across a variety of domains. It represents a significant step forward for multi-agent systems, achieving competitive performance on a number of agentic benchmarks (see the technical report for full details).

When originally released in November 2024 Magentic-One was implemented directly on the autogen-core library. We have now ported Magentic-One to use autogen-agentchat, providing a more modular and easier to use interface.

Using Magentic-One involves interacting with a digital world designed for humans, which carries inherent risks. To minimize these risks, consider the following precautions:

- Use Containers: Run all tasks in docker containers to isolate the agents and prevent direct system attacks.

- Virtual Environment: Use a virtual environment to run the agents and prevent them from accessing sensitive data.

- Monitor Logs: Closely monitor logs during and after execution to detect and mitigate risky behavior.

- Human Oversight: Run the examples with a human in the loop to supervise the agents and prevent unintended consequences.

- Limit Access: Restrict the agents’ access to the internet and other resources to prevent unauthorized actions.

- Safeguard Data: Ensure that the agents do not have access to sensitive data or resources that could be compromised. Do not share sensitive information with the agents. Be aware that agents may occasionally attempt risky actions, such as recruiting humans for help or accepting cookie agreements without human involvement. Always ensure agents are monitored and operate within a controlled environment to prevent unintended consequences. Moreover, be cautious that Magentic-One may be susceptible to prompt injection attacks from webpages.

## Example of using AI Assistant Agents & Magentic One Agents with the MAGENTIC ONE Orchestrator

## Import Required Dependencies

In [1]:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_ext.agents.file_surfer import FileSurfer
from autogen_ext.agents.magentic_one import MagenticOneCoderAgent
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_ext.teams.magentic_one import MagenticOne
import yaml


## Create a Model Client

In [2]:
with open("model_config.yaml", "r") as f:
        model_config = yaml.safe_load(f)
model_client = AzureOpenAIChatCompletionClient.load_component(model_config)

## Create Basic Team with one Assistant only

In [ ]:
async def main() -> None:
    assistant = AssistantAgent(
        "Assistant",
        model_client=model_client,
    )
    team = MagenticOneGroupChat([assistant], model_client=model_client)
    await Console(team.run_stream(task="Provide proof of the Pythagorean theorem using a diagram."))
    await model_client.close()


await main()


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:164: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:177: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (user) ----------
Provide proof of the Pythagorean theorem using a diagram.
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

Provide proof of the Pythagorean theorem using a diagram.


To answer this request we have assembled the following team:

Assistant: An agent that provides assistance with ability to use tools.


Here is an initial fact sheet to consider:

### 1. GIVEN OR VERIFIED FACTS  
- The request itself directly asks to provide proof of the Pythagorean theorem using a diagram.  
- The Pythagorean theorem states that in a right-angled triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides: \( a^2 + b^2 = c^2 \).  

### 2. FACTS TO LOOK UP  
- Historical methods for proving the Pythagorean theorem via geometric diagrams, if desired for contextual completeness (e.g., Euclidean proofs, visual proofs in ancient texts like "Elements" by Euclid). Thes

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)


---------- TextMessage (Assistant) ----------
I will create a diagram of the Pythagorean theorem proof using squares and triangles, known as a "visual proof." Let me proceed using a drawing tool to design the illustration.


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/agents/_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- TextMessage (MagenticOneOrchestrator) ----------
Proceed by designing a diagram for the Pythagorean theorem proof using squares and triangles. Include a step-by-step explanation of how the diagram illustrates the equation a^2 + b^2 = c^2.
---------- TextMessage (Assistant) ----------
I'll use a digital drawing tool to construct the diagram, illustrating how the squares formed on the sides of a right triangle satisfy the Pythagorean theorem. One moment, please!
---------- TextMessage (MagenticOneOrchestrator) ----------
Please create or find the requested diagram to visually prove the Pythagorean theorem and accompany it with a clear explanation of how it works.
---------- TextMessage (Assistant) ----------
I will employ an appropriate tool to construct or locate the diagram that visually demonstrates the Pythagorean theorem. Let me proceed now.
---------- TextMessage (MagenticOneOrchestrator) ----------
Create a clear visual proof of the Pythagorean theorem using square and 

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:428: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

Provide proof of the Pythagorean theorem using a diagram.


To answer this request we have assembled the following team:

Assistant: An agent that provides assistance with ability to use tools.


Here is an initial fact sheet to consider:

### Updated Fact Sheet

#### 1. GIVEN OR VERIFIED FACTS  
- The request directly asks to provide proof of the Pythagorean theorem using a diagram.  
- The Pythagorean theorem states that in a right-angled triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides: \( a^2 + b^2 = c^2 \).  
- A geometric or visual proof inherently focuses on using shapes like squares and triangles to demonstrate mathematical relationships.  

#### 2. FACTS TO LOOK UP  
- Specific examples of **visual proofs** of the Pythagorean theorem beyond classical Euclidean proofs, such as the "square rearrangement proof" or mode

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:440: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

Provide proof of the Pythagorean theorem using a diagram.


To answer this request we have assembled the following team:

Assistant: An agent that provides assistance with ability to use tools.


Here is an initial fact sheet to consider:

### Updated Fact Sheet  

#### 1. GIVEN OR VERIFIED FACTS  
- The request explicitly asks to provide proof of the Pythagorean theorem using a diagram.  
- The Pythagorean theorem states that in a right-angled triangle, the square of the hypotenuse is equal to the sum of the squares of the other two sides: \( a^2 + b^2 = c^2 \).  
- A geometric or visual proof inherently focuses on using shapes like squares and triangles to demonstrate mathematical relationships.  
- Visual proofs often use rearrangement or decomposition strategies to show area equivalency across the squares \( a^2, b^2,\) and \( c^2 \) drawn on the respective sides of t

## Create a team with a multimodal websurfer

- WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.

In [3]:
# The example code may download files from the internet, execute code, and interact with web pages. 
# Ensure you are in a safe environment before running the example code.
async def main() -> None:
    surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=model_client,
    )

    team = MagenticOneGroupChat([surfer], model_client=model_client)
    await Console(team.run_stream(task="What is the UV index in Toronto today?"))




await main()


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:164: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:177: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (user) ----------
What is the UV index in Toronto today?
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

What is the UV index in Toronto today?


To answer this request we have assembled the following team:

WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.


Here is an initial fact sheet to consider:

1. GIVEN OR VERIFIED FACTS  
- The request asks for the UV index specifically in Toronto.  

2. FACTS TO LOOK UP  
- The current UV index in Toronto.  
- This information can typically be found on weather websites (e.g., Weather.com, AccuWea

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:610: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- MultiModalMessage (WebSurfer) ----------
I typed 'current UV index Toronto' into '0 characters out of 2000'.

The web browser is open to the page [current UV index Toronto - Search](https://www.bing.com/search?q=current+UV+index+Toronto&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=7B1CE8BFCA734EB0861B9D1FA199E81D).
The viewport shows 31% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

By using this site you agree to the use of cookies for analytics, personalized content, and ads.
Learn more about third party cookies
|
Microsoft Privacy Policy 
Skip to content
current UV index TorontoFrançais3
MobileAll
Search
News
Images
Videos
Maps
Shopping
More
Tools
About 394,000 resultsRecommended for you:uv index toronto 
todaytoronto uv index 
forecasttoronto 
14 day forecast accuweathertoronto 
city uv index
toronto uv index 
hourlyuv index 
environment canada listuv index 
today delisletoronto 
weather map radar 
Toronto, Onta

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)


---------- TextMessage (MagenticOneOrchestrator) ----------
The UV index in Toronto today is 6, which is considered high. It’s a good idea to take precautions such as wearing sunscreen and protective clothing if you plan to be outdoors.


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:455: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


## Create a multi-agent team with the following capabilities

- WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.

- FileSurfer: An agent that can handle local files.

- Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.

- ComputerTerminal: A computer terminal that performs no other action than running Python scripts (provided to it quoted in python code blocks), or sh shell scripts (provided to it quoted in sh code blocks).



In [4]:

# The example code may download files from the internet, execute code, and interact with web pages. 
# Ensure you are in a safe environment before running the example code.

async def main() -> None:
    surfer = MultimodalWebSurfer(
        "WebSurfer",
        model_client=model_client,
    )

    file_surfer = FileSurfer( "FileSurfer",model_client=model_client)
    coder = MagenticOneCoderAgent("Coder",model_client=model_client)
    terminal = CodeExecutorAgent("ComputerTerminal",code_executor=LocalCommandLineCodeExecutor())
    
    team = MagenticOneGroupChat([surfer, file_surfer, coder, terminal], model_client=model_client)

    await Console(team.run_stream(task="What is the UV index in Melbourne today?"))



await main()


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:164: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:177: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (user) ----------
What is the UV index in Melbourne today?
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

What is the UV index in Melbourne today?


To answer this request we have assembled the following team:

WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.
FileSurfer: An agent that can handle local files.
Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.
ComputerTerminal: A computer terminal that performs no other action than running Python scripts (provide

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:610: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- MultiModalMessage (WebSurfer) ----------
I typed 'UV index Melbourne today' into '0 characters out of 2000'.

The web browser is open to the page [UV index Melbourne today - Search](https://www.bing.com/search?q=UV+index+Melbourne+today&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=2D737D3136504D959D53C0190367325F).
The viewport shows 28% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

By using this site you agree to the use of cookies for analytics, personalized content, and ads.
Learn more about third party cookies
|
Microsoft Privacy Policy 
Skip to content
UV index Melbourne todayFrançaisAll
Search
Images
Videos
Maps
News
Copilot
More
Tools
Recommended for you:live uv index melbourne
uv index melbourne 
nowmelbourne 
average monthly uv index
uv 
rating for today
uv 
rays melbourne today
uv index melbourne 
sunsmartsunsmart uv today
uv index 
forecast epa 
Copilot Answer
Melbourne, Victoria, Australia
04:41AM
Update

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)


---------- TextMessage (MagenticOneOrchestrator) ----------
The UV index in Melbourne today is currently 2, which is considered low.


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:455: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


## This uses Magentic_one helper class with all the agents bundled together



In [5]:



async def example_usage():
    client = model_client
    m1 = MagenticOne(client=client)
    task = "Write a Python script to fetch data from the OpenWeatherMap API, and save the file locally."
    result = await Console(m1.run_stream(task=task))
    print(result)


if __name__ == "__main__":
    await example_usage()


/var/folders/dj/qp0fwj152ks28q9cn0_rd3fw0000gn/T/ipykernel_78543/3879225610.py:3: DeprecationWarning: Instantiating MagenticOne without a code_executor is deprecated. Provide a code_executor to clear this warning (e.g., code_executor=LocalCommandLineCodeExecutor() ).
  m1 = MagenticOne(client=client)
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:164: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:177: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext

---------- TextMessage (user) ----------
Write a Python script to fetch data from the OpenWeatherMap API, and save the file locally.
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

Write a Python script to fetch data from the OpenWeatherMap API, and save the file locally.


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.
WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.
Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.
Co

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:610: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- MultiModalMessage (WebSurfer) ----------
I typed 'OpenWeatherMap API documentation endpoint authentication response format' into '0 characters out of 2000'.

The web browser is open to the page [OpenWeatherMap API documentation endpoint authentication response format - Search](https://www.bing.com/search?q=OpenWeatherMap+API+documentation+endpoint+authentication+response+format&form=QBLH&sp=-1&lq=0&pq=&sc=0-0&qs=n&sk=&cvid=8B368365AC844B58802274FD828B3286).
The viewport shows 24% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

By using this site you agree to the use of cookies for analytics, personalized content, and ads.
Learn more about third party cookies
|
Microsoft Privacy Policy 
Skip to content
OpenWeatherMap API documentation endpoint authentication response formatFrançais3
All
Search
Copilot
Videos
Images
Maps
News
Filters
About 424,000 resultsDetermines the 
format of the 
response. Possible values are xml an

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)


---------- MultiModalMessage (WebSurfer) ----------
I typed 'https://openweathermap.org/api' into the browser address bar.

The web browser is open to the page [Weather API - OpenWeatherMap](https://openweathermap.org/api).
The viewport shows 10% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

GuideAPIDashboardMarketplacePricingMapsOur InitiativesPartnersBlogFor BusinessSign inSupport centreWeather API
HomeWeather API
 Please, 
sign up to use our fast and easy-to-work weather APIs. As a start to use OpenWeather products, we recommend our 
One Call API 3.0. For more functionality, please consider our products, which are included in 
professional collections. 
One Call API 3.0API doc
Subscribe
Make an API call to receive access to the various data:
Current weather and forecasts: minute forecast for 1 hour
 hourly forecast for 48 hours
 daily forecast for 8 days
 and government weather alerts 
Weather data for any timestamp for 46+ 

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/agents/_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- TextMessage (MagenticOneOrchestrator) ----------
Here's the Python script to fetch weather data from the OpenWeatherMap API and save it locally:

```python
import requests
import json

def fetch_weather_data(api_key, latitude, longitude):
    # OpenWeatherMap API endpoint
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={api_key}"

    try:
        # Send request to OpenWeatherMap API
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code == 200:
            weather_data = response.json()

            # Save response locally as 'weather_data.json'
            with open("weather_data.json", "w") as json_file:
                json.dump(weather_data, json_file, indent=4)
            
            print("Weather data fetched and saved successfully as 'weather_data.json'.")
        else:
            print(f"Failed to fetch weather data. HTTP Status Code: {response.s

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:455: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
